In [71]:
import random
from santorinai.board import Board
from santorinai.pawn import Pawn
from santorinai.player import Player
from santorinai.player_examples.basic_player import BasicPlayer

Basic Rules

In [72]:
def movesPlayer(board, players, x, y):
    z = board[x][y]
    moves = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= len(board):
                continue
            if y+j < 0 or y+j >= len(board):
                continue
            if board[x+i][y+j] - z < 2 and board[x+i][y+j] < 4 and (x+i, y+j) not in players:
                moves.append((x+i, y+j))
    return moves

In [73]:
def constructsPlayer(board, players, x, y):
    places = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            if x+i < 0 or x+i >= len(board):
                continue
            if y+j < 0 or y+j >= len(board):
                continue
            if board[x+i][y+j] < 4 and (x+i, y+j) not in players:
                places.append((x+i, y+j))
    return places

In [74]:
def win(board, x, y):
    return board[x][y] == 3

Minimax

In [75]:
def minimax(board, players, playerAct, depth, alpha, beta, maximizingPlayer):
    if win(board, *players[(playerAct-1) % 4]):
        return -1 if maximizingPlayer else 1
    if depth == 0:
        return 0
    for move in movesPlayer(board, players, *players[playerAct]):
        newPlayers = players[:]
        newPlayers[playerAct] = move
        for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
            newBoard = [row[:] for row in board]
            newBoard[construct[0]][construct[1]] += 1
            value = minimax(newBoard, newPlayers, (playerAct+1) %
                            4, depth-1, alpha, beta, not maximizingPlayer)
            if maximizingPlayer:
                alpha = max(alpha, value)
                if alpha >= beta:
                    return alpha
            else:
                beta = min(beta, value)
                if beta <= alpha:
                    return beta
    return alpha if maximizingPlayer else beta

In [76]:
def meanimean(board, players, playerAct, depth, maximizingPlayer):
    if win(board, *players[(playerAct-1) % 4]):
        return -1 if maximizingPlayer else 1
    if depth <= 0:
        return 0
    sum = 0
    nbActions = 0
    for move in movesPlayer(board, players, *players[playerAct]):
        newPlayers = players[:]
        newPlayers[playerAct] = move
        for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
            newBoard = [row[:] for row in board]
            newBoard[construct[0]][construct[1]] += 1
            value = meanimean(newBoard, newPlayers, (playerAct+1) %
                            4, depth-1, not maximizingPlayer)
            sum += value
            nbActions += 1
    if nbActions == 0: #pion bloqué
        return 0.5 if maximizingPlayer else -0.5
    return sum / nbActions

In [77]:
def getBestMove(board, players, playerAct, depth):
    bestValue = -2
    bestMove = None
    bestConstruct = None
    bestMean = -2
    while bestMove == None:
        for move in movesPlayer(board, players, *players[playerAct]):
            newPlayers = players[:]
            newPlayers[playerAct] = move
            for construct in constructsPlayer(board, newPlayers, *newPlayers[playerAct]):
                if depth == 0:
                    return move, construct
                newBoard = [row[:] for row in board]
                newBoard[construct[0]][construct[1]] += 1
                value = minimax(newBoard, newPlayers, (playerAct+1) %
                                4, depth-1, bestValue, 1, False)
                if value != None and value > bestValue:
                    bestValue = value
                    bestMove = move
                    bestConstruct = construct
                    bestMean = meanimean(newBoard, newPlayers, (playerAct+1) % 4, depth-3, False)
                    if bestValue >= 1:
                        return bestMove, bestConstruct
                elif value == bestValue:
                    newMean = meanimean(newBoard, newPlayers, (playerAct+1) % 4, depth-3, False)
                    if newMean > bestMean:
                        bestMean = newMean
                        bestMove = move
                        bestConstruct = construct
        depth -= 1
    return bestMove, bestConstruct

In [78]:
import threading
import time

class threadWithReturn(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(threadWithReturn, self).__init__(*args, **kwargs)
        self._return = None
    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args, **self._kwargs)

def playProgressive(board, players, playerAct, depth):
    time_start = time.time()
    act_depth = 0
    while act_depth <= depth:
        thread = threadWithReturn(target=getBestMove, args=(board, players, playerAct, act_depth))
        thread.start()
        while time.time() - time_start <= 5 and thread.is_alive():
            time.sleep(0.1)
        if time.time() - time_start > 5:
            print(act_depth - 1)
            return move, construct
        act_depth += 1
        move, construct = thread._return
    print(depth)
    return move, construct

In [79]:
class TktBot3(Player):
    """
    Minimax + Monte Carlo Tree Search bot
    """

    def name(self):
        return "Tkt bot 3"

    # Placement of the pawns
    def place_pawn(self, board, pawn):
        my_choice = (2, 2) if (2, 2) in board.get_possible_movement_positions(
            pawn) else random.choice(board.get_possible_movement_positions(pawn))
        return my_choice

    # Movement and building
    def play_move(self, board, pawn):
        return playProgressive(
                board.board, [player.pos for player in board.pawns], pawn.number - 1, 20)

Test

In [80]:
from santorinai.tester import Tester
from santorinai.player_examples.random_player import RandomPlayer
from santorinai.player_examples.first_choice_player import FirstChoicePlayer
from santorinai.player_examples.basic_player import BasicPlayer
from TktBot2 import TktBot2

tester = Tester()
tester.delay_between_moves = 0
tester.display_board = False
tester.verbose_level = 2

p1 = TktBot3()
p2 = TktBot2()

tester.play_1v1(p1, p2, nb_games = 70)

Game 1
Player Tkt bot 2 is placing pawn 1
   Pawn placed at position (2, 2)
Player Tkt bot 3 is placing pawn 2
   Pawn placed at position (3, 1)
Player Tkt bot 2 is placing pawn 3
   Pawn placed at position (4, 3)
Player Tkt bot 3 is placing pawn 4
   Pawn placed at position (1, 0)

Playing the game
   Current pawn: pawn 1 of player 1 at position (2, 2)
Player Tkt bot 3 is moving pawn 1
5
   Pawn moved at position (1, 1) and built at position (0, 0)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 30 
_0 _0 _0 _0 _0 
_0 10 _0 20 _0 
_1 40 _0 _0 _0 

   Current pawn: pawn 2 of player 2 at position (3, 1)
Player Tkt bot 2 is moving pawn 2
6
   Pawn moved at position (2, 0) and built at position (2, 1)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 30 
_0 _0 _0 _0 _0 
_0 10 _1 _0 _0 
_1 40 20 _0 _0 

   Current pawn: pawn 3 of player 1 at position (4, 3)
Player Tkt bot 3 is moving pawn 3
5
   Pawn moved at position (3, 2) and built at position (2, 1)

_0 _0 _0 _0 _0 
_0 _0 _0 _0 _0 
_0 _0 _0 30 _0 
_0 10 _2 _0 _0 
_1 40 20 _0

{'Tkt bot 3': 33, 'Tkt bot 2': 37}